In [1]:
%matplotlib inline
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import optical_flow as op_flow
import ntu_rgb
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(linewidth=200)
import pandas as pd
import cv2
import scipy
from scipy import optimize

In [3]:
def show(im):
    plt.figure(figsize=(20, 15))
    plt.imshow(im)
    plt.show()

## Building training data

In [4]:
''' Get the dataset '''
dataset = ntu_rgb.NTU()
# dataset.load() # NOTE: only need to do once

metadata.pkl found in current path. Use this file? [Y/n] 


In [ ]:
# Make sure the metadata is correct:
for m in dataset.metadata:
    vid_num = m['video_index']
    if vid_num % 1000 != 0:
        continue
    print("video:", vid_num, "loss:", m['s_loss'])
    rgb_im   = dataset.get_rgb_vid_images(vid_num)
    depth_im = dataset.get_depth_images(vid_num)
    d_2_r    = dataset.get_depth_2_rgb_map(vid_num)
    new_rgb = np.zeros([depth_im.shape[1], depth_im.shape[2], 3])
    for d_coord, rgb_coord in d_2_r.items():
        new_rgb[d_coord[1], d_coord[0]] = rgb_im[0][int(rgb_coord[1]), int(rgb_coord[0])]
    show(new_rgb.astype(np.uint8))

In [5]:
vid_id = 0
rgb_3D = dataset.get_rgb_3D_maps(vid_id)
op_flow_2D = dataset.get_2D_optical_flow(vid_id)

In [16]:
print(len(rgb_3D))
print(np.nonzero(rgb_3D[0])[0].shape)
print(np.nonzero(rgb_3D[99])[0].shape)

103
(16068,)
(15414,)


In [ ]:
opflow_3D = dataset.get_3D_optical_flow(vid_id, rgb_3D, op_flow_2D)

In [ ]:
num_frames = len(opflow_3D)
max_vecs = max([len(o) for o in opflow_3D.values()])

opflow_ani = np.zeros([6,num_frames,max_vecs])
for frame in opflow_3D:
    for idx, vec in enumerate(opflow_3D[frame]):
        opflow_ani[:,frame,idx] = vec

opflow_ani[1,:,:] *= -1 # flip y axis
opflow_ani[4,:,:] *= -1 # flip y vectors
# opflow_ani[2,:,:] = np.power(opflow_ani[2,:,:], 3) # Increase spread of z values

In [ ]:
import ipyvolume.pylab as p3
print("shape of opflow data", opflow_ani.shape)
fig = p3.figure()
# p3.zlim(2,8)
q = p3.quiver(*opflow_ani[:,:,:], color="red", size=3)
p3.style.use("dark") # looks better
p3.animation_control(q, interval=100, add=True)
p3.show()
fig.animation = 0

In [ ]:
# def set_view(figure, framenr, fraction):
#     q.sequence_index = framenr
#     p3.view(fraction*360 + 0.5, 0)
# p3.movie('3D_opflow.mp4', set_view, frames=98*2, fps=20, gif_loop=0)

In [ ]:
# Visualize point cloud
vid_num = 6
point_clouds = dataset.get_point_clouds(vid_num)
from pyntcloud import PyntCloud
df_pointcloud = pd.DataFrame(point_clouds[50,:,:], columns=['x','y','z'])
cloud = PyntCloud(df_pointcloud)
cloud.plot()

In [ ]:
# Visualize first set of images
vid_num = 58
m = dataset.metadata[vid_num]
print(m['video_index'])
rgb_im   = dataset.get_rgb_vid_images(vid_num)
depth_im = dataset.get_depth_images(vid_num)
for frame in range(rgb_im.shape[0]):
    new_rgb = np.zeros([depth_im.shape[1], depth_im.shape[2], 3])
    d_2_r = dataset.get_depth_2_rgb_map(vid_num, frame)
    for d_coord, rgb_coord in d_2_r.items():
        new_rgb[d_coord[1], d_coord[0]] = rgb_im[frame][int(rgb_coord[1]), int(rgb_coord[0])]
    show(new_rgb.astype(np.uint8))

In [ ]:
# Visualize optical flow
vid_num = 58
images = dataset.get_rgb_vid_images(vid_num, True)
flow_maps = dataset.get_2D_flow_maps(vid_num)
ani = op_flow.get_animation(images, flow_maps)
from IPython.display import HTML
HTML(ani.to_html5_video())
# ani.save('2D_opflow.mp4', writer="ffmpeg")

In [ ]:
'''
Show epipolar line on an image
'''

def show_epipolar_line(skeleton_df, rgb_images, F):
    frame = skeleton_df[skeleton_df['frame_index'] == 0]
    im = rgb_images[0].copy()
    
    # Epipolar line (depth point -> color line)
    epi = F @ np.array([frame['depth'].iloc[3][0], frame['depth'].iloc[3][1], 1.])

    # Create epipolar line mask on RGB image
    indices = np.ones([3, im.shape[0], im.shape[1]])
    indices[:2,:,:] = np.indices([im.shape[0], im.shape[1]])
    epi_mat = abs((indices[1] * epi[0]) + (indices[0] * epi[1]) + epi[2])
    line_vals = epi_mat < 0.01

    # Put red values on image 
    red_mat = np.zeros_like(im)
    red_mat[1,:,:] = 255
    im = np.where(np.stack([line_vals]*3,2), red_mat, im)

    for x in range(-5,5):
        for y in range(-5,5):
            im[int(frame['color'].iloc[3][1]) + y, int(frame['color'].iloc[3][0]) + x] = np.array([0,255,0])

    show(im)

In [ ]:
#  RGB Intrinsic Parameters
fx_rgb = 1.0820918205955327e+03
fy_rgb = 1.0823759977994873e+03
cx_rgb = 9.6692449993169430e+02
cy_rgb = 5.3566594698237566e+02

# Depth Intrinsic Parameters
fx_d = 365.481
fy_d = 365.481
cx_d = 257.346
cy_d = 210.347

# Distortion parameters
k1 = 0.0905474
k2 = -0.26819
k3 = 0.0950862
p1 = 0.
p2 = 0.

# Rotation [depth-to-color] ; should this be transposed?
R = -np.array([[9.9997086703565385e-01, -5.6336988734456616e-03, -5.1503899819367671e-03],
               [5.6034256549301270e-03,  9.9996705123920560e-01, -5.8735046520488696e-03],
               [5.1833098395106967e-03,  5.8444737120895603e-03,  9.9996948724755408e-01]])

# 3D Translation (meters)
t_x = 5.2236787502888557e-02
t_y = -3.0543659328634320e-04
t_z = -9.6233443531043941e-04

# Matrix versions
t_array = np.array([t_x, t_y, t_z])
rgb_mat = np.array([[fx_rgb,     0., cx_rgb],
                    [    0., fy_rgb, cy_rgb],
                    [    0.,     0.,     1.]])
d_mat   = np.array([[fx_d,   0., cx_d],
                    [  0., fy_d, cy_d],
                    [  0.,   0.,   1.]])
dist_array = np.array([k1, k2, k3, p1, p2])

In [ ]:
'''
Old functions. todo: replace
'''

def find_scale_term(skeleton_df, R, T):
    frame = skeleton_df[skeleton_df['frame_index'] == 0]
    c_2D_gt = frame.iloc[3]['color']

    # Convert depth to 3D:
    u = frame['depth'].iloc[3][0]
    v = frame['depth'].iloc[3][1]
    z = frame['loc'].iloc[3][2]
    d_3D = np.linalg.inv(d_mat) @ np.array([u, v, 1]) * z

    # Convert 3D to rgb:
    def loss_func(s):
        rgb_3D = (R @ d_3D.reshape(1,-1).T) + T*s
        c_2D = ((rgb_mat @ rgb_3D) / rgb_3D[2]).T[0][:2]
        return np.linalg.norm(c_2D.T - c_2D_gt)
    
    opt_dict = scipy.optimize.minimize(loss_func, 0)
    loss = opt_dict['fun']
    scale = opt_dict['x'][0]
    print("Scale: {} \t (loss = {})".format(scale, loss))
    return scale

def get_r_t(skeleton_df):
    objectPoints = np.array([skeleton_df['loc']]).astype('float32')
    rgb          = np.array([skeleton_df['color']]).astype('float32')
    d            = np.array([skeleton_df['depth']]).astype('float32')

    # Fundamental matrix
    F, mask = cv2.findFundamentalMat(d, rgb)

    # Essential matrix
    E = rgb_mat.T @ F @ d_mat

    # Decompose essential matrix
    R1, R2, T = cv2.decomposeEssentialMat(E)

    # Get the rotation matrix that looks more like the identity matrix
    if abs(np.sum(R1 - np.identity(3))) < abs(np.sum(R2 - np.identity(3))):
        R = R1
    else:
        R = R2

    # print("Fundamental matrix:\n", F)
    # print("Essential matrix:\n", E)
    print("Rotation matrix:\n", R)
    print("Translation (unit) d->rgb: ", T.T)
    return R, T, F

def rgb_2_depth(rgb, depth, R, T, s):
    new_rgb = np.zeros([depth.shape[0], depth.shape[1], 3])
    for y in range(depth.shape[0]):
        for x in range(depth.shape[1]):            
            ''' Set to black if nothing in depth '''
            if depth[y,x] == 0:
                new_rgb[y, x] = np.array([0,0,0])
                continue
            
            ''' Convert to meters '''
            z = depth[y,x] / 1000
            
            ''' Depth --> Camera coordinates '''
            x_3D = (x - cx_d) * z / fx_d
            y_3D = (y - cy_d) * z / fy_d
            
            ''' Apply rotation and translation '''
            p0 = np.array([x_3D, y_3D, z])
            p = (R @ p0.reshape(1,-1).T) + T*s

            ''' Camera coordinates --> RGB '''
            x_rgb = (p[0] * rgb_mat[0,0] / p[2]) + rgb_mat[0,2]
            y_rgb = (p[1] * rgb_mat[1,1] / p[2]) + rgb_mat[1,2]
            
            if y_rgb >= 1080:
                continue
            if x_rgb >= 1920:
                continue
            new_rgb[y, x] = rgb[int(y_rgb), int(x_rgb)]
    return new_rgb.astype(np.uint8)

In [ ]:
full_df = pd.DataFrame()
# for vid in dataset.metadata[:200]:
#     skeleton_df = dataset.get_skeleton_data(vid['video_index'])
#     full_df = full_df.append(skeleton_df)


first_vid = dataset.metadata[7000]['video_set']
skeleton_dfs = []
for idx, vid in enumerate(dataset.metadata):
    if vid['video_set'][:2] == first_vid[:2]:
        print(idx)
        skeleton_dfs.append(dataset.get_skeleton_data(vid['video_index']))

full_df = pd.concat(skeleton_dfs, ignore_index=True)
R, T, F = get_r_t(full_df)

videos_shown = []
for index, m in full_df.iterrows():
    vid_num = m['video_index']
    if vid_num in videos_shown:
        continue
    else:
        videos_shown.append(vid_num)
    skeleton_df = dataset.get_skeleton_data(vid_num)
    rgb_images  = dataset.get_rgb_vid_images(vid_num)
    d_images    = dataset.get_depth_images(vid_num)
    mins = find_scale_term(skeleton_df, R, T)
    im = rgb_2_depth(rgb_images[0], d_images[0], R, T, mins)
    show(im)

In [ ]:
# import pandas as pd
# from pyntcloud import PyntCloud
# df_pointcloud = pd.DataFrame(point_clouds[0,:,:], columns=['x','y','z'])
# cloud = PyntCloud(df_pointcloud)
# cloud.plot()

In [ ]:
ir_images  = dataset.get_rgb_vid_images(0)
flow_maps = dataset.get_2D_flow_maps(ir_images)
ani = op_flow.get_animation(ir_images, flow_maps)
from IPython.display import HTML
HTML(ani.to_html5_video())

In [ ]:
# ir_images  = dataset.get_ir_vid_images(0)
# ir_images[:,:40,:] = 0
# ir_images[:,-40:,:] = 0
# ir_images[:,:,:40] = 0
# ir_images[:,:,-40:] = 0
# flow_maps = dataset.get_2D_flow_maps(ir_images)
# ani = op_flow.get_animation(ir_images, flow_maps)
# from IPython.display import HTML
# HTML(ani.to_html5_video())

In [ ]:
# Save animation

# ani.save('optical_flow_ir.mp4', writer="ffmpeg")

In [ ]:


for x in range(5):
    vid_num = x*1234
    rgb_images = dataset.get_rgb_vid_images(vid_num)
    depth_images = dataset.get_depth_images(vid_num)
    rgb_depth = rgb_2_d(rgb_images[0], depth_images[0])
    show(rgb_depth)